## Aim: To model the cost impact of dispensing 2 or 3 months rather than single months

We examined the cost of a policy switch to recommend two or three monthly prescriptions across the NHS where there is no clinical rationale for issuing shorter durations. 


In [1]:
import os
from ebmdatalab import bq
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
from ipywidgets import Layout

# \*** Scroll down to "Model" for the fully-formed model. 

Earlier sections describe reasoning and set out the detail behind the calculations

### What are the costs to include?

> Longer prescription lengths were associated with more medication waste per prescription. However, when including dispensing fees and prescriber time, longer prescription lengths resulted in lower TUC. This finding was consistent across all five cohorts. Savings ranged from £8.38 to £12.06 per prescription per 120 days if a single long prescription was issued instead of multiple short prescriptions. Prescriber time costs accounted for the largest component of TUC.

[Doble et al 2017]

**1. Dispensing fees**

- Currently 126p per item. 
- Plus 2% of the cost per prescription (cost per day multiplied by prescription length) for prescriptions over £100 \*see below

(https://psnc.org.uk/dispensing-supply/endorsement/fees-allowances/) 

*Fees for expensive prescriptions (over £100)*

Not a consideration for the drugs we are looking at, all of which are available generically:
- Ramipril £2-5 /pack https://openprescribing.net/tariff/?codes=0205051R0AAADAD&codes=0205051R0AAANAN&codes=0205051R0AAAAAA&codes=0205051R0AAAKAK&codes=0205051R0AAABAB&codes=0205051R0AAALAL&codes=0205051R0AAACAC&codes=0205051R0AAAMAM
- Atorvastatin <£2/pack https://openprescribing.net/tariff/?codes=0212000B0AAAAAA&codes=0212000B0AAABAB&codes=0212000B0AAACAC&codes=0212000B0AAADAD
- Simvastatin <£2/pack https://openprescribing.net/tariff/?codes=0212000Y0AAAAAA&codes=0212000Y0AAABAB&codes=0212000Y0AAADAD&codes=0212000Y0AAAHAH
- Levothyroxine max £15/pack during 18-19 https://openprescribing.net/tariff/?codes=0602010V0AABZBZ&codes=0602010V0AAGHGH&codes=0602010V0AABWBW&codes=0602010V0AABXBX&codes=0602010V0AABYBY
- Amlodipine <£2/pack https://openprescribing.net/tariff/?codes=0206020A0AAABAB&codes=0206020A0AAAAAA

**2. Prescriber time**

Depending on the item, the prescriber time per prescription varies [Doble et al 2017]
- antihypertensives: £3.77 (in 3-month scenario), £3.76 (28 days)	
- diabetes: £3.55 (in 3-month scenario), £3.54 (28 days)	
- SSRIs: £3.18 (in 3-month scenario), £3.23 (28 days)

Note they use Figures from Curtis et al 2015, which are:
- GP time £3.80/min, all practice expenses and including qualification costs (excluding quals it would be £3.20/min) - however, these are costs per minute **of patient contact** i.e. denominator used to calculate this rate was total patient contact time rather than total working hours?  
- GP Nurse time £0.93/min (£56/hr per hour of F2F contact time), including qualification costs. (Basic rate is £36/hr / 0.6/min)

Latest GP/nurse Figures from Curtis et al 2019:
- GP  £132/hr (£2.20/min)
- Nurse £37/hr (£0.62/min)

With electronic repeat _dispensing_, it is possible that 12 months' of 28+ days' supply are authorised in one go, which means that changing the duration of each prescription would have no effect on prescriber time. With repeat _prescribing_, the patient must still request each prescription from the practice. 

**3. Wastage**

Depending on the item, the longer the duration, the more medicines will be wasted [Doble et al 2017]
- antihypertensives: £0.51 (in 3-month scenario), £0.07 (28 days)	
- diabetes: £1.37 (in 3-month scenario), £0.33 (28 days)	
- SSRIs: £0.43 (in 3-month scenario), £0.21 (28 days)

- For statins (secondary prevention) they found that 3.325% days of shorter duration 
  (under 60 days) prescription supplies were wasted,
  compared to 3.663% of longer duration (>=60 days) prescriptions. 

**4. Patient time and expenses**
- Time required to travel to pharmacy / GP?
- DIfferences for dispensing practices (>1.6m)?
- Time taken to collect prescription? Under electronic prescribing prescriptions may already be ready to collect when the patient arrives. 

- Sliding scale might be the best way! 
- The majority of population live within 20 mins walk of a pharmacy (few years old maybe)

**5. Income generation**
- Patients eligible to pay for prescriptions pay a fixed amount per prescription which contributes towards the total prescribing bill. For each prescription dispensed for 84 rather than 28 days, income would be reduced by 2/3.

- However, the majority of prescriptions (90%?) are exempt from charges, for example over 60s, under 19s, people on low incomes etc. Those eligible to pay who collect a lot of medications can have their total capped. 

- Therefore, for the medications we are considering we may expect that >95% of the prescriptions (2x statins, largely over-60s) are not paid for by patients. 

- Crucially, where patients pay for routine prescriptions, it is likely that they are already given 84 days supply to reduce the cost burden on them.

- Therefore, we would expect a very small decrease in income from changing patients from 28 day prescriptions to longer durations.

**Refs**
- *Doble B, Payne R, Harshfield A, Wilson EC. BMJ Open. 2017;7(12):e019382.*
- *Curtis L , Burns A . Unit Costs of Health and Social Care. Canterbury, UK: Personal Social Services Research Unit, The University of Kent, 2015*

**Example from previous literature**

> **Three-Month Versus 28-Day Prescribing of Antihypertensives**

> We conducted two analyses representing alternative scenarios. 
In both cases we first added in the transaction and drug wastage costs (Table 3). 

> For the 3-month arm this equated to an extra 
£21.01 per annum [= **dispensing fees** (£0.90) + **prescriber time** (£3.77) + **wastage costs** (£0.51) × (365/90)], 

> and in the 28-day arm, 
£61.68 [= dispensing fees (£0.90) + prescriber time (£3.76) + wastage costs (£0.07) × (365/28)].

From Martin A, Payne R, Wilson EC. Appl Health Econ Health Policy 2018;16:317–30. doi:10.1007/s40258-018-0383-9

## Load summary prescribing data

In [2]:
df = pd.read_csv(os.path.join('..','data','table1_qpi_summary.csv'))
items = df.set_index('quantity_per_item')['items']
display(items)

quantity_per_item
28    88410515
56    39714191
84     4834340
Name: items, dtype: int64

## We assume that approx 10% of one-month prescriptions are appropriate (e.g. newly initiated patients). 

# Calculations - formation of model

## 1. Dispensing fees

- 126p - cost per item dispensed (not divided across items on same prescription form)


In [3]:
dispensing = 1.26

dispensing_28 = dispensing*items[28]

display(Markdown(f"Current total dispensing cost for all 28-day supplies: **£{dispensing_28/1E6:.1f} M**"))

# Calculate cost of dispensing 90% of 28-day prescriptions in 3-month batches:
n=0.9*dispensing_28/3

display(Markdown(
    f"Potential reduction in cost if 90% one-month prescriptions were 3-months: **£{n/1E6:.1f} M** ({100*n/dispensing_28:.1f}% of total 28-day dispensing fees)"
    ))

Current total dispensing cost for all 28-day supplies: **£111.4 M**

Potential reduction in cost if 90% one-month prescriptions were 3-months: **£33.4 M** (30.0% of total 28-day dispensing fees)

## 2. Prescriber time

In [4]:
cdoc = 2.2 # cost per minute for GP time
cnur = 0.62 # cost per minute for nurse time

# note the values for the following variables are selected arbitrarily
prop_doc = 0.5 # proportion approved by a GP
prop_nur = 1-prop_doc # proportion approved by a nurse

t = 0.5 # time taken to approve a repeat prescription (minutes)
prop_erd = 0.5 # proportion of prescriptions on electronic repeat dispensing (assume zero cost - or one cost per 12 months?)

cpp_doc = cdoc*t*(1-prop_erd) # cost per average prescription (doc)
cpp_nur = cnur*t*(1-prop_erd) # cost per average prescription (nur)

approval = (prop_doc*cpp_doc) + (prop_nur*cpp_nur)
approval_28 = items[28]*approval # cost to approve all 28-day prescriptions

display(Markdown(f"Current total estimated approval cost for all 28-day supplies: **£{approval_28/1E6:.1f} M**"),
       Markdown(f"Potential reduction in cost if 90% were 84-days: **£{(0.9*approval_28/3)/1E6:.1f} M** ({100*(0.9*approval_28/3)/approval_28:.1f}% of total 28-day approval fees)"
    ))

Current total estimated approval cost for all 28-day supplies: **£31.2 M**

Potential reduction in cost if 90% were 84-days: **£9.3 M** (30.0% of total 28-day approval fees)

## 3. Wastage

In [5]:
waste_s = 0.03325 # 3.325% days wasted of shorter duration (\<60 days) prescription supplies
waste_l = 0.03663 # 3.663% of longer duration  (\>=60 days) prescriptions. 
# [Doble et al 2017]

display(Markdown("#### Example: Atorvastatin (For price estimate, 20mg tabs averaged around 80p per 28-tablet pack in 2018-19)"))
p = 0.8
print ("£%8.5f"%(p*waste_s), " (3.325%) wasted per 28-day supply")
print ("£%8.5f"%(3*p*waste_l), " (3.663%) wasted per 84-day supply")

# Assuming this scales up to all of our 5 medicines...
display(Markdown("#### Scaling up using 80p per item estimate"))

waste_28 = p*items[28]*waste_s

print ("£{:,.2f}".format(waste_28/1E6), "M (3.325%) estimated current wastage of all 28-day supplies")
print ("£{:,.2f}".format((p*waste_l*items[28]*3*0.9)/1E6 + (p*waste_s*items[28]*0.1)/1E6), "M wasted if 90% of all 28-day supplies were 84-day")

#### Example: Atorvastatin (For price estimate, 20mg tabs averaged around 80p per 28-tablet pack in 2018-19)

£ 0.02660  (3.325%) wasted per 28-day supply
£ 0.08791  (3.663%) wasted per 84-day supply


#### Scaling up using 80p per item estimate

£2.35 M (3.325%) estimated current wastage of all 28-day supplies
£7.23 M wasted if 90% of all 28-day supplies were 84-day


## 4. Patient time and expenses

In [6]:
# Conservatively assume 10 mins per prescription (a low estimate to account for most people going to the pharmacy while nearby)
t = 10/60 # hours per prescription
print ("{:,.1f}".format((t*12)), " estimated hours per patient per year on 28-day supplies")
print ("{:,.1f}".format((t*12/3)), " estimated hours per patient per year on 84-day supplies")

cost_public = 11 # cost of public time per hour
total_time_28 = t*items[28]
total_time_switch = ((0.1)+(0.9/3))*total_time_28
display(Markdown (f"{total_time_28/1E6:,.0f} M estimated total patient hours to collect all 28-day supplies (**£{cost_public*total_time_28/1E6:.1f}M** at £{cost_public}/h)"),
        Markdown (f"{total_time_switch/1E6:,.0f} M total patient hours if 90% 28-day supplies were 84 days (**£{cost_public*total_time_switch/1E6:.1f}M** at £{cost_public}/h)"))



2.0  estimated hours per patient per year on 28-day supplies
0.7  estimated hours per patient per year on 84-day supplies


15 M estimated total patient hours to collect all 28-day supplies (**£162.1M** at £11/h)

6 M total patient hours if 90% 28-day supplies were 84 days (**£64.8M** at £11/h)

# Model
 This calculates the total costs for 28-day prescriptions assuming X% are converted to 2/3/4 month supplies.
 (It does not add on the cost for existing 56 and 84 day supplies)

In [7]:
#### Common Variables
prescriptions = items[28] # total 28-day prescriptions
percent_amenable = 0.9 # proportion of prescriptions amenable to change to longer durations 0.5-0.95
months_supply = 3 # 2-4


##### Dispensing fees
dispensing = 1.26
# Calculate cost of dispensing 90% of prescriptions in 3-month batches

def calculate_dispensing_fees(prescriptions, months_supply, percent_amenable, dispensing):
    return prescriptions* (# no of items
           dispensing) * ( # cost multiplier
           (1-percent_amenable)+(percent_amenable/months_supply)) # adjusted number of items


##### Staff Cost
prop_doc = 0.5 # proportion approved by a GP
t = 0.5 # time taken to approve a repeat prescription (minutes)
prop_erd = 0.5 # proportion of prescriptions on electronic repeat dispensing (assume zero cost - or one cost per 12 months?)

def calculate_staff_cost(prescriptions, months_supply, percent_amenable, prop_doc, prop_erd, t):
    cdoc = 2.2 # cost per minute for GP time
    cnur = 0.62 # cost per minute for nurse time
    prop_nur = 1-prop_doc # proportion approved by a nurse
    cpp_doc = cdoc*t*(1-prop_erd) # cost per average prescription (doc)
    cpp_nur = cnur*t*(1-prop_erd) # cost per average prescription (nur)
    
    return prescriptions*( # no of items
           prop_doc*cpp_doc + prop_nur*cpp_nur)*( # cost multiplier
           (1-percent_amenable)+(percent_amenable/months_supply)) # adjusted number of items


##### Wastage
priceperitem = 0.8 # average cost per box (28-day supply)

def calculate_waste(prescriptions, months_supply, percent_amenable, priceperitem):
    waste_s = 0.03325
    waste_l = 0.03663
    return prescriptions*(   # no of items
            priceperitem)*(  # cost multiplier
                (waste_l*percent_amenable*months_supply) + (# waste for longer prescriptions 
                # note here we multiply by months_supply rather than divide as the amount of product subject to potential waste will increase with longer duration
                waste_s*(1-percent_amenable))) # waste for remaining 28 day prescriptions


#Patient Cost
cost_public = 11 # cost of public time per hour
time_collect = 10 # time to collect prescription (min) 

def calculate_patient_cost(prescriptions, months_supply, percent_amenable, cost_public, time_collect):
    return prescriptions*( # no of items
           cost_public*time_collect/60)*(  # cost multiplier
           (1-percent_amenable)+(percent_amenable/months_supply)) # adjusted number of items
           



# Apply Model

In [8]:
percent_amenable = 0.9   

model_input = [1,2,3] # current status vs switch to 2 or 3 months

model_output = pd.DataFrame(index=["dispensing","staff","waste","patient"], columns=[1,2,3])


for months_supply in model_input: 
    
    dispensing_fees = calculate_dispensing_fees(prescriptions, months_supply, percent_amenable, dispensing)    
    staff_cost = calculate_staff_cost(prescriptions, months_supply, percent_amenable, prop_doc, prop_erd, t)
    waste = calculate_waste(prescriptions, months_supply, percent_amenable, priceperitem)
    patient_cost = calculate_patient_cost(prescriptions, months_supply, percent_amenable, cost_public, time_collect)

    model_output[months_supply] = [dispensing_fees/1E6, staff_cost/1E6, waste/1E6, patient_cost/1E6]
    
    
    display_text =f"Estimated cost for 28-day prescriptions with {percent_amenable*100}% switched to **{months_supply} month** supplies: "
    if months_supply==1:
        display_text = "Current estimated cost for 28-day prescriptions: "
    display(Markdown(f"{display_text} **£{(dispensing_fees + staff_cost + waste + patient_cost) /1E6:.1f} M** ")
           )

# calculate total and savings    
model_output = model_output.transpose()
model_output["total"] = model_output.sum(axis=1)
model_output["saving"] = model_output["total"][1] - model_output["total"]

model_output.round(1).to_csv(os.path.join('..','data','cost_model_summary.csv'))
display(model_output.round(1))

Current estimated cost for 28-day prescriptions:  **£307.2 M** 

Estimated cost for 28-day prescriptions with 90.0% switched to **2 month** supplies:  **£172.5 M** 

Estimated cost for 28-day prescriptions with 90.0% switched to **3 month** supplies:  **£129.1 M** 

,dispensing,staff,waste,patient,total,saving
1,111.4,31.2,2.6,162.1,307.2,0.0
2,61.3,17.1,4.9,89.1,172.5,134.8
3,44.6,12.5,7.2,64.8,129.1,178.1


## Extract prescribing data at CCG level

In [9]:
### extract 28-day prescribing data for modelling

sql = '''
SELECT
  pct,
  SUM(IF(quantity_per_item=28,items,0)) AS items_28d,
  SUM(items) AS total_items,
  SUM(IF(quantity_per_item=28,net_cost,0)) AS net_cost_28d

FROM
 ebmdatalab.hscic.raw_prescribing_normalised AS presc
INNER JOIN  hscic.ccgs AS ccgs ON presc.pct=ccgs.code AND ccgs.org_type='CCG'

WHERE
quantity_per_item IN (28,56,84)
AND month BETWEEN '2018-08-01' AND '2019-07-01'
AND 
(bnf_code LIKE "0205051R0%" OR  ##ramipril
bnf_code LIKE "0212000B0%" OR ##atrovastatin
bnf_code LIKE "0212000Y0%" OR ##simvastatin
bnf_code LIKE "0602010V0%" OR ##levothyroxine
bnf_code LIKE "0206020A0%") ##amlodipine
AND
(bnf_name LIKE '%_Tab%' or bnf_name LIKE '%_Cap%') ##this restricts to tablets or capsules

GROUP BY pct
    '''

df_ltc = bq.cached_read(sql, csv_path=os.path.join("..", "data", "ltc_qty_cost.csv"))

df_ltc.head()

,pct,items_28d,total_items,net_cost_28d
0,01C,427787,564865,428006.32
1,02D,300760,336528,296100.18
2,12F,636722,884347,761271.54
3,00C,87656,206447,92475.54
4,01G,361738,532114,364012.25


In [10]:
# calculate cost per item

data = df_ltc.set_index("pct").sort_index()
# add a total row
data = data.append(data.sum().rename("All")).reset_index()

# calculate additional fields
data["percent_28d"] = 100*data['items_28d']/data['total_items']
data["cost_per_item"] = data['net_cost_28d']/data['items_28d']

data.tail()

,pct,items_28d,total_items,net_cost_28d,percent_28d,cost_per_item
187,99J,938302.0,1147677.0,956260.99,81.756627,1.019140
188,99K,220987.0,381550.0,232927.40,57.918228,1.054032
189,99M,182687.0,423717.0,179975.66,43.115334,0.985159
190,99N,826270.0,1221770.0,833533.07,67.628932,1.008790
191,All,87207906.0,131173827.0,90824270.97,66.482703,1.041468


## Output with widgets to adjust variables

In [11]:
# ensure tests pass despite conflict with widget output:
# NBVAL_IGNORE_OUTPUT 

prescriptions = data.loc[data["pct"]=="All", "items_28d"].item() 
priceperitem = data.loc[data["pct"]=="All", "cost_per_item"].item()

#ccg_selector = widgets.Dropdown(options=data[["pct","items"]].to_numpy().tolist(), value=prescriptions, description='CCG:', disabled=False)
ccg_selector = widgets.Dropdown(options=data["pct"], value="All", description='CCG:', disabled=False)

# make widget titles fit by setting the style
style = {'description_width': 'initial'}


months_supply_slider = widgets.IntSlider(min=1, max=4, step=1, description='Months supply:', style=style, value=3)

# proportion of prescriptions amenable to change
percent_amenable_slider = widgets.FloatSlider(min=0.2, max=0.95, step=0.05, description='Proportion amenable:', style=style, value=0.9)

# proportion of prescriptions approved by a GP
prop_doc_slider = widgets.FloatSlider(min=0.1, max=0.9, step=0.1, description='Proportion GP:', style=style, value=0.5)

# time taken to approve a repeat prescription (minutes)
time_slider = widgets.FloatSlider(min=0.1, max=2, step=0.1, description='Time per presc (min):', style=style, value=0.5)

# proportion of prescriptions on electronic repeat dispensing (assume zero cost - or one cost per 12 months?)
prop_erd_slider = widgets.FloatSlider(min=0, max=1, step=0.1, description='e-RD proportion:', style=style, value=0.5)

# price per item
#priceperitem_slider = widgets.FloatSlider(min=0.5, max=2, step=0.1, description='Price per item (£):', style=style, value=0.8)

# cost of public time per minute
cost_public_slider = widgets.IntSlider(min=5, max=15, step=1, description='Cost of public time per hour (£):', style=style, value=11)

# time to collect prescription (minutes)
time_collect_slider = widgets.IntSlider(min=0, max=30, step=1, description='Time to collect (min):', style=style, value=10)

    
def f(ccg_selector, months_supply_slider, percent_amenable_slider, prop_doc_slider, time_slider, prop_erd_slider, 
      cost_public_slider, time_collect_slider):
    
    ccg = ccg_selector
    months_supply = months_supply_slider
    percent_amenable = percent_amenable_slider
    prop_doc = prop_doc_slider
    t = time_slider
    prop_erd = prop_erd_slider
    #priceperitem = priceperitem_slider
    cost_public = cost_public_slider
    time_collect = time_collect_slider
    
    # items and cost per item, using ccg from dropdown
    prescriptions = data.loc[data["pct"]==ccg, "items_28d"].item() 
    percent_28d = data.loc[data["pct"]==ccg, "percent_28d"].item() 
    priceperitem = data.loc[data["pct"]==ccg, "cost_per_item"].item()

    
    dispensing_fees = calculate_dispensing_fees(prescriptions, months_supply, percent_amenable, dispensing)    
    staff_cost = calculate_staff_cost(prescriptions, months_supply, percent_amenable, prop_doc, prop_erd, t)
    waste = calculate_waste(prescriptions, months_supply, percent_amenable, priceperitem)
    patient_cost = calculate_patient_cost(prescriptions, months_supply, percent_amenable, cost_public, time_collect)

    print(f'Total 28-day prescriptions = {prescriptions:,.0f} \n'
          f'Percent 1 month (of all 1+2+3 month prescriptions) = {percent_28d:,.0f}% \n'
          f'Mean price per item = £{priceperitem:,.2f} \n'
          f'\n'
          f'Dispensing fees =  £{dispensing_fees/1E6:,.1f} M \n'
          f'Staff cost =  £{staff_cost/1E6:,.1f} M \n'
          f'Wasted meds =  £{waste/1E6:,.1f} M \n'
          f'Patient cost =  £{patient_cost/1E6:,.1f} M \n \n'
          f'Overall impact: =  £{(staff_cost+ waste+ patient_cost)/1E6:,.1f} M'
         )

out = widgets.interactive_output(f, 
                                 {'ccg_selector': ccg_selector,
                                  'months_supply_slider': months_supply_slider, 
                                  'percent_amenable_slider': percent_amenable_slider,
                                  'prop_doc_slider': prop_doc_slider, 
                                  'time_slider': time_slider, 
                                  'prop_erd_slider': prop_erd_slider,
                                  #'priceperitem_slider': priceperitem_slider, 
                                  'cost_public_slider': cost_public_slider, 
                                  'time_collect_slider': time_collect_slider})


widgets.HBox([widgets.VBox([months_supply_slider, percent_amenable_slider, prop_doc_slider, time_slider, prop_erd_slider, 
      cost_public_slider, time_collect_slider, ccg_selector, out])])

In [12]:
# Display static output
f(ccg_selector="All", 
  months_supply_slider = 3, 
  percent_amenable_slider = 0.9, 
  prop_doc_slider = 0.5, 
  time_slider=0.5, 
  prop_erd_slider=0.5, 
  cost_public_slider=11, 
  time_collect_slider=10)

Total 28-day prescriptions = 87,207,906 
Percent 1 month (of all 1+2+3 month prescriptions) = 66% 
Mean price per item = £1.04 

Dispensing fees =  £44.0 M 
Staff cost =  £12.3 M 
Wasted meds =  £9.3 M 
Patient cost =  £64.0 M 
 
Overall impact: =  £85.5 M


## Apply model to larger basket of drugs
**Note larger basket includes 2/3-per-day drugs so we will infer the proportion 28-day from the smaller basket, per CCG.**

In [13]:
larger_basket = pd.read_csv(os.path.join('..','data','data_cost_model.csv'))
larger_basket.head() 

,Unnamed: 0,pct,chemical,chemical_code,total_items,total_tabs_caps,ltc_stablish
0,316,15E,Metformin Hydrochloride,0601022B0,464973.0,51957468.0,1.0
1,317,15N,Metformin Hydrochloride,0601022B0,428195.0,36868913.0,1.0
2,318,15M,Metformin Hydrochloride,0601022B0,367099.0,36240550.0,1.0
3,319,15C,Metformin Hydrochloride,0601022B0,323889.0,29016221.0,1.0
4,320,06H,Metformin Hydrochloride,0601022B0,367565.0,28887270.0,1.0


In [14]:
lb = larger_basket.copy()

# proportion 28d from small basket for each ccg:
lb = lb.merge(data[["pct","percent_28d"]], on="pct")

lb.head()

,Unnamed: 0,pct,chemical,chemical_code,total_items,total_tabs_caps,ltc_stablish,percent_28d
0,316,15E,Metformin Hydrochloride,0601022B0,464973.0,51957468.0,1.0,54.048758
1,1279,15E,Atorvastatin,0212000B0,793295.0,31884539.0,1.0,54.048758
2,1586,15E,Levothyroxine Sodium,0602010V0,479433.0,22148795.0,1.0,54.048758
3,1883,15E,Ramipril,0205051R0,477518.0,19808243.0,1.0,54.048758
4,2181,15E,Amlodipine,0206020A0,513319.0,21414652.0,1.0,54.048758
